In [36]:
import datetime

import backtrader

import lstm

In [37]:
# datafeed to provide indexes to frames to be used by a strategy
class FrameFeed(backtrader.feed.DataBase):
    params = (
        ("fromdate", datetime.datetime),
        ("todate", datetime.datetime),
        ("frames", None),
    )

    lines = ("datetime", "frame_index", "close")

    def __init__(self):
        super()

    def start(self):
        super(backtrader.feed.DataBase, self).start()
        # TODO: use assignment expressions
        self.frames = (
            frame for frame in enumerate(self.p.frames)
            if self.p.fromdate <= frame[1][-1][-1][0] <= self.p.todate
        )

    def _load(self):
        try:
            i, frame = next(self.frames)
        except StopIteration:
            return False

        print(f"next frame: {frame}")

        self.lines.datetime[0] = backtrader.utils.date2num(frame[-1][-1][0])
        self.lines.frame_index[0] = i

        return True

In [38]:
# strategy to use by backtesting
class TestStrategy(backtrader.Strategy):
    params = (
        ("model", None),
        ("frames", [[[]]]),
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt, txt))

    def __init__(self):
        self.frame_index = self.datas[1].frame_index
        self.frames = self.params.frames
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def next(self):
        if self.order:
            return

        current_frame_index = self.frame_index[0]
        previous_frame_index = self.frame_index[-1]
        current_frame = [[col[1] for col in vector] for vector in self.frames[int(current_frame_index)]]
        previous_frame = [[col[1] for col in vector] for vector in self.frames[int(previous_frame_index)]]

        current_normalized_frame = lstm.normalize_frame(current_frame)
        previous_normalized_frame = lstm.normalize_frame(previous_frame)

        current_prediction = lstm.predict_sequences_multiple(self.p.model, [current_normalized_frame])
        ##previous_prediction = lstm.predict_sequences_multiple(self.p.model, [previous_normalized_frame])

        diff = current_prediction[0][0] - previous_normalized_frame[-1][-1]
        # diff = current_prediction[0][0] - previous_prediction[0][0]
        # diff = current_prediction[0][0] - current_normalized_frame[-1][0]

        if self.position and diff < 0:
            # self.log(f"SELL: {current_prediction} <= {previous_prediction}")
            self.order = self.sell()
        else:
            if diff > 0:
                # self.log(f"BUY: {current_prediction} > {previous_prediction}")
                self.order = self.buy()

In [39]:
# setup inital testing strategy
cerebro = backtrader.Cerebro()
cerebro.broker.setcash(100000.0)
cerebro.addsizer(backtrader.sizers.SizerFix, stake=20)
cerebro.broker.setcommission(commission=0.005)

In [40]:
# get testing data
times = lstm.get_time_series_daily("MSFT", ["4. close"], outputsize="full")
vectors = lstm.times_to_vectors(times, include_time=True)[::-1]

train_vectors, test_vectors = lstm.partition_data(vectors)

train_frames = lstm.get_frames(train_vectors, 15, with_target=True)
test_frames = lstm.get_frames(test_vectors, 15, with_target=False)

train_no_dates = [[[col[1] for col in vector] for vector in frame] for frame in train_frames]
normalized_train = lstm.normalize_frames(train_no_dates)
x_train, y_train = lstm.seperate_xy(normalized_train)

In [41]:
# setup model
model = lstm.setup_lstm_model(x_train, y_train)

compilation time :  0.021867036819458008
Train on 4031 samples, validate on 213 samples
Epoch 1/1
4031/4031 [==============================] - 3s 626us/step - loss: 1.0000 - val_loss: 0.7717


In [42]:
# add data and model to strategy
cerebro.addstrategy(TestStrategy, model=model, frames=test_frames)

cerebro.adddata(
    backtrader.feeds.YahooFinanceData(
        dataname='MSFT',
        fromdate=datetime.datetime(2018, 1, 1),
        todate=datetime.datetime(2019, 1, 1)
    )
)

cerebro.adddata(
    FrameFeed(
        frames=test_frames,
        fromdate=datetime.datetime(2018, 1, 1),
        todate=datetime.datetime(2019, 1, 1)
    )
)

In [43]:
print(f'Beginning Portfolio Value: {cerebro.broker.getvalue()}')

cerebro.run()

print(f'Final Portfolio Value: {cerebro.broker.getvalue()}')

Beginning Portfolio Value: 100000.0
next frame: [[(datetime.datetime(2017, 12, 11, 0, 0), '85.2300')], [(datetime.datetime(2017, 12, 12, 0, 0), '85.5800')], [(datetime.datetime(2017, 12, 13, 0, 0), '85.3500')], [(datetime.datetime(2017, 12, 14, 0, 0), '84.6900')], [(datetime.datetime(2017, 12, 15, 0, 0), '86.8500')], [(datetime.datetime(2017, 12, 18, 0, 0), '86.3800')], [(datetime.datetime(2017, 12, 19, 0, 0), '85.8300')], [(datetime.datetime(2017, 12, 20, 0, 0), '85.5200')], [(datetime.datetime(2017, 12, 21, 0, 0), '85.5000')], [(datetime.datetime(2017, 12, 22, 0, 0), '85.5100')], [(datetime.datetime(2017, 12, 26, 0, 0), '85.4000')], [(datetime.datetime(2017, 12, 27, 0, 0), '85.7100')], [(datetime.datetime(2017, 12, 28, 0, 0), '85.7200')], [(datetime.datetime(2017, 12, 29, 0, 0), '85.5400')], [(datetime.datetime(2018, 1, 2, 0, 0), '85.9500')]]
next frame: [[(datetime.datetime(2017, 12, 12, 0, 0), '85.5800')], [(datetime.datetime(2017, 12, 13, 0, 0), '85.3500')], [(datetime.datetime(20

2018-01-03, BUY EXECUTED, Price: 84.24, Cost: 1684.80, Comm 8.42
2018-01-04, BUY EXECUTED, Price: 84.75, Cost: 1695.00, Comm 8.47
2018-01-05, BUY EXECUTED, Price: 85.80, Cost: 1716.00, Comm 8.58
2018-01-08, BUY EXECUTED, Price: 86.33, Cost: 1726.60, Comm 8.63
2018-01-09, BUY EXECUTED, Price: 86.77, Cost: 1735.40, Comm 8.68
2018-01-10, BUY EXECUTED, Price: 86.00, Cost: 1720.00, Comm 8.60
2018-01-11, BUY EXECUTED, Price: 86.26, Cost: 1725.20, Comm 8.63
2018-01-12, BUY EXECUTED, Price: 86.79, Cost: 1735.80, Comm 8.68
2018-01-16, BUY EXECUTED, Price: 88.19, Cost: 1763.80, Comm 8.82
2018-01-17, BUY EXECUTED, Price: 87.19, Cost: 1743.80, Comm 8.72
2018-01-18, BUY EXECUTED, Price: 87.90, Cost: 1758.00, Comm 8.79
2018-01-19, BUY EXECUTED, Price: 88.23, Cost: 1764.60, Comm 8.82
2018-01-22, BUY EXECUTED, Price: 88.09, Cost: 1761.80, Comm 8.81
2018-01-23, BUY EXECUTED, Price: 89.95, Cost: 1799.00, Comm 9.00
2018-01-24, SELL EXECUTED, Price: 90.59, Cost: 1737.84, Comm 9.06
2018-01-25, BUY EXECUTED

2018-07-05, Order Canceled/Margin/Rejected
2018-07-06, Order Canceled/Margin/Rejected
2018-07-06, Order Canceled/Margin/Rejected
2018-07-09, Order Canceled/Margin/Rejected
2018-07-09, Order Canceled/Margin/Rejected
2018-07-10, Order Canceled/Margin/Rejected
2018-07-10, Order Canceled/Margin/Rejected
2018-07-11, Order Canceled/Margin/Rejected
2018-07-11, Order Canceled/Margin/Rejected
2018-07-12, Order Canceled/Margin/Rejected
2018-07-12, Order Canceled/Margin/Rejected
2018-07-13, Order Canceled/Margin/Rejected
2018-07-13, Order Canceled/Margin/Rejected
2018-07-16, Order Canceled/Margin/Rejected
2018-07-16, Order Canceled/Margin/Rejected
2018-07-17, SELL EXECUTED, Price: 103.32, Cost: 1805.81, Comm 10.33
2018-07-18, SELL EXECUTED, Price: 104.63, Cost: 1805.81, Comm 10.46
2018-07-19, BUY EXECUTED, Price: 103.64, Cost: 2072.80, Comm 10.36
2018-07-20, SELL EXECUTED, Price: 106.75, Cost: 1810.76, Comm 10.68
2018-07-23, BUY EXECUTED, Price: 104.99, Cost: 2099.80, Comm 10.50
2018-07-24, SELL 

2018-12-19, Order Canceled/Margin/Rejected
2018-12-19, Order Canceled/Margin/Rejected
2018-12-20, Order Canceled/Margin/Rejected
2018-12-20, Order Canceled/Margin/Rejected
2018-12-21, Order Canceled/Margin/Rejected
2018-12-21, Order Canceled/Margin/Rejected
2018-12-24, Order Canceled/Margin/Rejected
2018-12-24, Order Canceled/Margin/Rejected
2018-12-26, Order Canceled/Margin/Rejected
2018-12-26, Order Canceled/Margin/Rejected
2018-12-27, Order Canceled/Margin/Rejected
2018-12-27, Order Canceled/Margin/Rejected
2018-12-28, Order Canceled/Margin/Rejected
2018-12-28, Order Canceled/Margin/Rejected
2018-12-31, Order Canceled/Margin/Rejected
Final Portfolio Value: 111941.05699999996
